In [3]:
import pandas as pd
import numpy as np
import gc
import pickle
from matplotlib import pyplot as plt
from datetime import datetime
import time

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%matplotlib inline
import seaborn as sns

from sklearn import metrics

from sklearn.model_selection import cross_validate, GridSearchCV
from sklearn.model_selection import train_test_split
pd.set_option('display.max_columns', None)
sns.set()

ModuleNotFoundError: No module named 'pandas'

Given that the dataset contains recordings of vitals for 30 patients, to make the data easy to handle, we first created a list of dataframes where each dataframe contained per patient data

In [2]:
## a list to store the data for each patient 
dfs_list = []

We first start by loading the data...

In [ ]:
f = open('./dataset_OSAS.pickle', 'rb')
data = pickle.load(f)
f.close()

Here we parse the entire data, remove duplicates and initialize the list of dataframes

In [ ]:
## dropping duplicates from the data 
for i in range(1,31,1):
    df = data[data['patient'] == str(i)]
    df.drop_duplicates(subset=['timestamp_datetime'], inplace=True)
    dfs_list.append(df)

As a first step towards EDA, we first wanted to see different statistics associated with each feature for our better understanding of the nature of the data we are handling

In [ ]:
data.describe()

We first plotted the number of events per patient i.e apnea and hypopnea to see how the distribution events across subjects

In [ ]:
### plotting counts of hypoapnea events for each patient 
hypopnea_list = []

for i in range(1,31,1):
    temp = data[data['patient'] == str(i)]
    temp = temp['event'].value_counts()
    temp = pd.DataFrame(temp)
    temp.reset_index(inplace = True)
    hypoapnea = temp.loc[temp['index'] == 'HYPOPNEA']['event'].iloc[0]
    hypopnea_list.append(hypoapnea)

plt.rcParams["figure.figsize"] = [10, 7]
plt.rcParams["figure.autolayout"] = True
x_vals = range(1,31,1)
plt.bar(x_vals, hypopnea_list, color = 'green')
plt.xticks(x_vals)
plt.xlabel("Subject Number")
plt.ylabel("Hypopnea Count")
plt.title("Number of Hypopnea Event for each Patient")

In [ ]:
apnea_list = []
print(len(dfs_list))

for i in range(0,30,1):
    temp = dfs_list[i].query("event != 'HYPOPNEA'").query("event != 'NONE'")
    apnea_list.append(len(temp))

plt.rcParams["figure.figsize"] = [10, 7]
plt.rcParams["figure.autolayout"] = True
x_vals = range(1,31,1)
plt.bar(x_vals, apnea_list, color = 'green')
plt.xticks(x_vals)
plt.xlabel("Subject Number")
plt.ylabel("Apnea count")
plt.title("Number of Apnea Events for each Patient")

Since we had the recording of a number of vitals in the dataset, it seemed like a good idea to find the relationship between different features. First, we found out the covariance in the dataset between all the numeric values.However, ad can be seen from the table, we were not able to find a statiscially significant result

In [ ]:
data.cov()

Next, we created a correlation matrix to see if any correlations exist

In [ ]:
## finding the duration of time for each patient
count = 1
list_duration = []
for df in dfs_list:
    temp = list(df['timestamp_datetime'])
    t1 = temp[0]
    t2 = temp[-1]
    duration = round((t2 - t1).total_seconds() / 3600,1)
    list_duration.append(duration)

plt.rcParams["figure.figsize"] = [10, 7]
plt.rcParams["figure.autolayout"] = True
x_vals = range(1,31,1)
plt.plot(x_vals, list_duration, color = 'purple')
plt.xticks(x_vals)
plt.xlabel("Subject Number")
plt.ylabel("Duration of recording (hrs)")
plt.title("Duration of recording for each patient in hours")

As can be seen, there is a significant difference in the duration of recording between the subjects, especially for subject 6 and 12. Later as we analyse the data, we will explore further how that factors into the detection of apnea events.

Next we attempt to get an understanding of data ranges.

In [ ]:
features = ['RR(rpm)', 'HR(bpm)', 'SpO2(%)', 'PI(%)', 'PVCs(/min)']

fig, ax = plt.subplots(nrows=len(features), ncols=1, figsize=(20,20))

## For each feature for the each patient, do a box plot
for index in range(len(features)):
    ax[index].set_title(features[index])
    
    patient_data = []
    for patient_id in np.unique(data['patient']):
        temp = data[data['patient'] == patient_id][features[index]]
        patient_data.append([x for x in list(temp.values) if not np.isnan(x)] )
    
    ax[index].boxplot(patient_data)

plt.show()

In [ ]:
## Plot an hour of data for each of the features
## If you notice, the plots for SpO2 and PI% don't begin until around 2100 for patient 1
## The values for the first portions of the data are NaNs so the data is missing
## The fact that there is missing data can be seen more clearly in the bottom table where there is
##   approximately 8% data missing for both of these features.
## I believe that these two measurements are often collected via the same device so it is likely that
##   the NaNs are place values for the time prior to hooking up the device.
## In addition, the red markings on the time series plot indicates hypopnea events marked by a clinician
##   while the purple marking indicates an apnea event. Idealy we would be able to see some correlation.

## Takes a set of patient data (assumes only a single patient) and a start and end then returns
##    a list of start and ending tuples. You can also specify the types it expects a list of
##    the valid types such as ['APNEA-CENTRAL' 'APNEA-MIXED' 'APNEA-OBSTRUCTIVE' 'HYPOPNEA' 'NONE']
def get_list_of_events(df, start, end, event_type):
    
    ## Will be in the format of [ (start1, end1), (start2, end2)] ]
    event_list = []
    
    current_start = 0
    
    ## State identifier if we are inside of an event range or out of it
    in_event = 0
    
    for i in range(start, end + 1):
        ## We are in an event if it is both the correct type and we have encountered a 1
        if df['anomaly'][i] == 1 and in_event == 0 and df['event'][i] in event_type:
            in_event = 1
            current_start = i
        elif df['anomaly'][i] == 1 and in_event == 1 and df['event'][i] not in event_type:
            ## If we are in an event and the type switches we have to stop
            in_event = 0
            event_list.append( (current_start, i - 1) )
        elif df['anomaly'][i] == 0 and in_event == 1:
            in_event = 0
            event_list.append( (current_start, i - 1) )
    
    return event_list
        
fig, ax = plt.subplots(nrows=len(features), ncols=1, figsize=(20,20))

## For each feature for the first patient, graph a time series over the first hour of monitoring
for index in range(len(features)):
    ax[index].set_title(features[index])
    
    ## Only take the first hour of data
    plot_data = data[data['patient'] == '1'][features[index]].head(3600)
    
    ax[index].plot(list(range(0,3600)), plot_data, color='b')
    
    apnea_events = get_list_of_events(data[data['patient'] == '1'], 0, 3600, ['APNEA-CENTRAL','APNEA-MIXED','APNEA-OBSTRUCTIVE'])
    hypopnea_events = get_list_of_events(data[data['patient'] == '1'], 0, 3600, ['HYPOPNEA'])
    
    for start, end in apnea_events:
        ax[index].axvspan(start, end, color="purple", alpha=0.3)
        
    for start, end in hypopnea_events:
        ax[index].axvspan(start, end, color="red", alpha=0.3)
    
plt.show()

In [ ]:
## Calculate missing data %

## Some of these patients have significant chunks of missing data primarily in regards to SpO2 and PI%
##  Given how much is missing we may not be able to depend too heavily on these features for training our model.

patient_missing = {}

for feat in features:

    patient_missing[feat] = []
    
    for patient_id in np.unique(data['patient']):
        temp = data[data['patient'] == patient_id][feat]
        vals = list(temp.values)
        missing = len([x for x in vals if np.isnan(x)] ) / len(vals)
        patient_missing[feat].append(missing)

df = pd.DataFrame(patient_missing, index = np.unique(data['patient']))
display(df)

In [ ]:
# Chose three random patients to show 
pt_nums = ['9', '17', '21']
events = ['PSG_Flow', 'PSG_Abdomen', 'PSG_Position', 'PSG_Snore', 'PSG_Thorax']

In [ ]:
for pt_num in pt_nums:
    patient = data[data['patient'] == pt_num]
    time = patient['timestamp_datetime']
    event = patient['event']
    
    for e in events:

        psg = patient[e]

        # collecting the data in an easier form to plot
        huge_arr = []
        time_arr = []
        event_arr = []
        for i in event.index:
            event_arr = np.append(event_arr, event[i])
            huge_arr = np.append(huge_arr, psg[i])
            time_arr = np.append(time_arr, time[i])

        # setting up plots -- 4000 was a number of points I chose that I thought 
        # didn't look too crowded in the space allocated for the plot
        num_plots = int(len(time_arr)/4000)
        fig, axs = plt.subplots(num_plots)
        fig.set_figheight(10)
        fig.set_figwidth(20)
        fig.tight_layout()
        start = 0
        end = 4000
        for i in range(num_plots):
            # find the hypopnea and apnea events
            hypopnea = (event_arr[start:end] == 'HYPOPNEA')
            apnea = (event_arr[start:end] == 'APNEA-OBSTRUCTIVE')
            # plot the time series
            axs[i].plot(time_arr[start:end], huge_arr[start:end])
            # plot the hypopnea and apnea events
            for j in range(len(hypopnea)):
                if hypoapnea[j] == True:
                    axs[i].axvline(time_arr[start:end][j], color='red', alpha=0.1)
            for j in range(len(apnea)):
                if apnea[j] == True:
                    axs[i].axvline(time_arr[start:end][j], color='purple', alpha=0.1)
            start += 4000
            end += 4000

        axs[0].set_title(str(e) + " for Patient "+ str(pt_num)+", Red = Hypopnea, Purple = Apnea")
        axs[num_plots-1].set_xlabel("Time")

In [ ]:
ecg_events = ['signal_ecg_i', 'signal_ecg_ii', 'signal_ecg_iii']

In [ ]:
#signal_ecg_i

for pt_num in pt_nums:
    patient = data[data['patient'] == pt_num]
    time = patient['timestamp_datetime']
    event = patient['event']
    
    # collecting the data in an easier form to plot
    for e in ecg_events:

        ecg = patient[e]

        huge_arr = []
        time_arr = []
        event_arr = []
        for i in event.index:
            event_arr = np.append(event_arr, event[i])
            huge_arr = np.append(huge_arr, ecg[i])
            time_arr = np.append(time_arr, time[i])

        # setting up plots -- 4000 was a number of points I chose that I thought 
        # didn't look too crowded in the space allocated for the plot
        num_plots = int(len(time_arr)/4000)
        fig, axs = plt.subplots(num_plots)
        fig.set_figheight(10)
        fig.set_figwidth(20)
        fig.tight_layout()
        start = 0
        end = 4000
        for i in range(num_plots):
            # find the hypopnea and apnea events
            hypopnea = (event_arr[start:end] == 'HYPOPNEA')
            apnea = (event_arr[start:end] == 'APNEA-OBSTRUCTIVE')
            # plot the time series
            axs[i].plot(time_arr[start:end], huge_arr[start:end])
            # plot the hypopnea and apnea events
            for j in range(len(hypoapnea)):
                if hypoapnea[j] == True:
                    axs[i].axvline(time_arr[start:end][j], color='red', alpha=0.1)
            for j in range(len(apnea)):
                if apnea[j] == True:
                    axs[i].axvline(time_arr[start:end][j], color='purple', alpha=0.1)
            start += 4000
            end += 4000

        axs[0].set_title(str(e) + " for Patient "+ str(pt_num)+", Red = Hypopnea, Purple = Apnea")
        axs[num_plots-1].set_xlabel("Time")

In [ ]:
sns.violinplot(np.log(data['RR(rpm)']))

In [ ]:
sns.violinplot(np.log(data['HR(bpm)']))

In [ ]:
sns.violinplot(np.log(data['SpO2(%)']))

In [ ]:
sns.distplot(data['HR(bpm)'])

In [ ]:
sns.distplot(data['SpO2(%)'])

In [ ]:
num_cols=dataset.columns
dataset.corr() 

In [ ]:
dataset.corr('spearman')

In [ ]:
df = pd.DataFrame(dataset, columns=['RR(rpm)', 'HR(bpm)', 'SpO2(%)', 'PI(%)', 'PVCs(/min)'])

df.plot.area(stacked=False);

In [ ]:
from pandas.plotting import scatter_matrix

df = pd.DataFrame(dataset, columns=['RR(rpm)', 'HR(bpm)', 'SpO2(%)', 'PI(%)'])

scatter_matrix(df, alpha=0.2, figsize=(6, 6), diagonal="kde");